In [1]:
rm(list = ls())
options(stringsAsFactors = FALSE)
library('Seurat')
library('ggplot2')
library("DEP")
library("dplyr")

Warning message:
"package 'Seurat' was built under R version 3.5.3"
Warning message:
"package 'ggplot2' was built under R version 3.5.3"
Warning message:
"package 'DEP' was built under R version 3.5.2"
Warning message in fun(libname, pkgname):
"mzR has been built against a different Rcpp version (1.0.0)
than is installed on your system (1.0.3). This might lead to errors
when loading mzR. If you encounter such issues, please send a report,
including the output of sessionInfo() to the Bioc support forum at 
https://support.bioconductor.org/. For details see also
https://github.com/sneumann/mzR/wiki/mzR-Rcpp-compiler-linker-issue."
Warning message:
"package 'dplyr' was built under R version 3.5.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
file <- '../../data//20150718_Cerberus_proteinGroups.txt'
ed_file <-'../../data//profile_experimental_design.csv'

data <- read.csv(file, sep = "\t", stringsAsFactors = FALSE)
experimental_design <- read.table(ed_file, header=TRUE, sep="\t", stringsAsFactors = FALSE)
experimental_design <- read.table(ed_file, header=TRUE, sep=";", stringsAsFactors = FALSE)
experimental_design$X <- NULL

In [3]:
data <- filter(data, Reverse != "+", Potential.contaminant != "+")
data_unique <- make_unique(data,"Majority.protein.IDs" ,"Gene.names",delim = " ")

In [6]:
type = "Intensity.NOT_"
LFQ_columns <- grep(type, colnames(data_unique))
data_se <- make_se(data_unique, LFQ_columns, experimental_design)

data_filt <- filter_missval(data_se, thr = 0)

In [7]:
data_norm <- normalize_vsn(data_filt)
data_imp <- impute(data_norm, fun = "MinProb", q = 0.01)

Loading required package: imputeLCMD

Warning message:
"package 'imputeLCMD' was built under R version 3.5.3"
Loading required package: tmvtnorm

Warning message:
"package 'tmvtnorm' was built under R version 3.5.3"
Loading required package: mvtnorm

Warning message:
"package 'mvtnorm' was built under R version 3.5.3"
Loading required package: Matrix

Warning message:
"package 'Matrix' was built under R version 3.5.3"
Loading required package: stats4

Loading required package: gmm

Warning message:
"package 'gmm' was built under R version 3.5.3"
Loading required package: sandwich

Warning message:
"package 'sandwich' was built under R version 3.5.3"
Loading required package: norm

Warning message:
"package 'norm' was built under R version 3.5.2"
Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:parallel':


[1] 1.389606


In [14]:
data_as_table <- data_imp@colData

In [18]:
pbmc <- CreateSeuratObject(data_as_table)

Warning message in storage.mode(from) <- "double":
"NAs introduced by coercion"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"


In [ ]:
pbmc[["percent.mt"]] <- PercentageFeatureSet(object = pbmc, pattern = "^MT-")
VlnPlot(object = pbmc, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)
plot1 <- FeatureScatter(object = pbmc, feature1 = "nCount_RNA", feature2 = "percent.mt")
plot2 <- FeatureScatter(object = pbmc, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
CombinePlots(plots = list(plot1, plot2))
pbmc <- subset(x = pbmc, subset = nFeature_RNA > 200 & nFeature_RNA < 10000 & percent.mt < 5)
pbmc <- NormalizeData(object = pbmc, normalization.method = "LogNormalize", scale.factor = 10000)
pbmc <- FindVariableFeatures(object = pbmc, selection.method = "vst", nfeatures = 2000)
all.genes <- rownames(x = pbmc)
pbmc <- ScaleData(object = pbmc, features = all.genes)
pbmc <- RunPCA(object = pbmc, features = VariableFeatures(object = pbmc))
pbmc <- FindNeighbors(object = pbmc, dims = 1:10)
pbmc <- FindClusters(object = pbmc, resolution = 0.5)
head(x = Idents(object = pbmc), 5)


In [19]:
FindMarkers(data_as_so, ident.1 = 1, min.pct = 0.25)

ERROR: Error in WhichCells.Seurat(object = object, idents = ident.1): Cannot find the following identities in the object: 1


In [4]:
rownames(raw_counts)<-raw_counts[,1]
raw_counts<-raw_counts[,-1]
colnames(raw_counts)<-raw_counts[1,]
raw_counts<-raw_counts[-1,]
pbmc <- CreateSeuratObject(raw_counts, min.cells = 3, min.features = 200, project = "mydata_scRNAseq")
pbmc[["percent.mt"]] <- PercentageFeatureSet(object = pbmc, pattern = "^MT-")
VlnPlot(object = pbmc, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)
plot1 <- FeatureScatter(object = pbmc, feature1 = "nCount_RNA", feature2 = "percent.mt")
plot2 <- FeatureScatter(object = pbmc, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
CombinePlots(plots = list(plot1, plot2))
pbmc <- subset(x = pbmc, subset = nFeature_RNA > 200 & nFeature_RNA < 10000 & percent.mt < 5)
pbmc <- NormalizeData(object = pbmc, normalization.method = "LogNormalize", scale.factor = 10000)
pbmc <- FindVariableFeatures(object = pbmc, selection.method = "vst", nfeatures = 2000)
all.genes <- rownames(x = pbmc)
pbmc <- ScaleData(object = pbmc, features = all.genes)
pbmc <- RunPCA(object = pbmc, features = VariableFeatures(object = pbmc))
pbmc <- FindNeighbors(object = pbmc, dims = 1:10)
pbmc <- FindClusters(object = pbmc, resolution = 0.5)
head(x = Idents(object = pbmc), 5)
pbmc <- RunUMAP(object = pbmc, dims = 1:10)
new.cluster.ids <- c("CD8T","1","2","3","4","5", "6", "7", "8","9","CD8T","11","12","13")
names(x = new.cluster.ids) <- levels(x = pbmc)
pbmc <- RenameIdents(object = pbmc, new.cluster.ids)
DimPlot(object = pbmc, reduction = "tsne", label = TRUE, pt.size = 0.5) + NoLegend()
table(Idents(object = pbmc))
pbmc<-subset(x = pbmc, idents = c("CD8T"))
cluster0.markers <- FindMarkers(object = pbmc, ident.1 = 0, min.pct = 0.25)
head(x = cluster0.markers, n = 20)
cluster_cols<-cluster_cols<-c("CD69","SELL","CD3E","PTPRC","CCR7","CD28","IL7R","B3GAT1","CD8A","CD27","IL2RA")
pbmc <- RunPCA(object = pbmc, features = cluster_cols)
pbmc <- FindNeighbors(object = pbmc, dims = 1:10)
pbmc <- FindClusters(object = pbmc, resolution = 0.1)
pbmc <- RunUMAP(object = pbmc, dims = NULL,features = cluster_cols)
DimPlot(object = pbmc, reduction = "umap", label = TRUE, pt.size = 0.5) + NoLegend()

ERROR: Error in eval(expr, envir, enclos): object 'raw_counts' not found
